In [24]:
import numpy as np
import pandas as pd
import pymannkendall as mk
from scipy.stats import poisson
import datetime as dt
from scipy import stats
import pickle
import warnings
warnings.filterwarnings('ignore')
monthly = pd.read_csv('Data\overall_monthly.csv')
sponsorCoverage = pd.read_csv('Data\sponsor_covgrp_all_monthly.csv')
Overall = pd.read_csv('Data\sample_data_agg_cleaned.csv')
test = pd.read_csv('Data\ChubbTest.csv')
data = monthly
#data = sponsorCoverage
#data = test
data['Month Name-Year']=pd.to_datetime(data['Accd_Month']+data['Accd_year'].astype(str),format='%B%Y')
data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')
#data['Month Name-Year']=pd.to_datetime(data['qtr2'].astype(str),format='%Y-%m')
#data['timestamp'] = data['Month Name-Year'].dt.strftime('%Y-%m')
#data = data.drop('Month Name-Year', 1)

In [25]:
data.columns.tolist()

['Accd_year',
 'Accd_Month',
 'claims_count',
 'exposure',
 'Ult_Claim_incurred',
 'Ult_Earned_Premium',
 'Attritional_Loss_Ratio',
 'Claims_Frequency',
 'Claims_Severity',
 'Average_Premium',
 'Month Name-Year',
 'timestamp']

In [26]:
#Setting Bounds and setting up user defined KPI's
#Num is short for numerator, Den is short for denominator, KPI will be used as the column header and name for the KPI
upperBound = 2
lowerBound = -2
threshold = 24

field1 = 'Sponsor_name'

#Column Names
#['Accd_year', 'Accd_Month', 'claims_count', 'exposure', 'Ult_Claim_incurred', 'Ult_Earned_Premium', 
#'Attritional_Loss_Ratio', 'Claims_Frequency', 'Claims_Severity', 'Average_Premium', 'Month Name-Year', 'timestamp']

L1KPI = 'ALR'
L1Num = 'Ult_Claim_incurred'
L1Den = 'Ult_Earned_Premium'

L2KPI1 = 'Severity'
L2Num1 = 'Ult_Claim_incurred'
L2Den1 = 'claims_count'

L2KPI2 = 'Frequency'
L2Num2 = 'claims_count'
L2Den2 = 'exposure'

L2KPI3 = 'Average_Premium'
L2Num3 = 'Ult_Earned_Premium'
L2Den3 = 'polcovg_count'

In [27]:
#Calculate values for KPI's for each row
#timeArray is used for the rolling average function. Creates an index with length defined by the threshold variable
data = data.replace('#VALUE!', 0)
data = data.replace('#DIV/0!', 0)
data[L1KPI] = data[L1Num]/data[L1Den]
data[L2KPI1] = data[L2Num1]/data[L2Den1]
data[L2KPI2] = data[L2Num2]/data[L2Den2]
#data[L2KPI3] = data[L2Num3]/data[L2Den3]

#timeArray is used for the rolling average function. Creates an index with length defined by the threshold variable
timelist = list(range(0,threshold,1))
timeArray = np.array(timelist)
len(timeArray)
data = data.fillna(0)

In [28]:
group = list(data[field1].unique())
groups = []
for c in group:
    locals()[c] = data.loc[data[field1] == c]
    groups.append(locals()[c])

for s in groups:
    Array = s['ALR'].to_numpy()
    Trend = []
    for i in range(0, len(Array), 1):
        if i>(threshold-1):
            x = timeArray
            y = Array[timeArray-threshold+i]
            m, b = np.polyfit(x, y, 1)
            Trend.append(b)
        else:
            Trend.append(0)

    Trend = np.asarray(Trend)
    s['Trend'] = Trend
data = pd.concat(groups).fillna(0)

KeyError: 'Sponsor_name'

In [31]:
#Takes the data in 24 row chunks (as defined by threshold) and... 
#calculates a slope using the numpy theilslopes function for each chunkArray = data['ALR'].to_numpy()
Array = data['ALR'].to_numpy()
Trend = []
for i in range(0, len(Array), 1):
    if i>(threshold-2):
        x = timeArray
        y = Array[timeArray-(threshold-1)+i]
        res = stats.theilslopes(y, x, 0.90)
        Trend.append(res[0])
    else:
        Trend.append(0)
        
Trend = np.asarray(Trend)
data['Trend'] = Trend

In [32]:
data['Change'] = data.Trend.shift(5) - data.Trend

In [33]:
data

,Accd_year,Accd_Month,claims_count,exposure,Ult_Claim_incurred,Ult_Earned_Premium,Attritional_Loss_Ratio,Claims_Frequency,Claims_Severity,Average_Premium,Month Name-Year,timestamp,ALR,Severity,Frequency,Trend,Change
0,2016,January,240,27288.51817,800016.3700,4312489.006,0.185512,0.008795,3333.401542,13.037609,2016-01-01,2016-01,0.185512,3333.401542,0.008795,0.000000,NaN
1,2016,February,266,25476.93156,826803.5300,4305321.276,0.192042,0.010441,3108.283947,13.044570,2016-02-01,2016-02,0.192042,3108.283947,0.010441,0.000000,NaN
2,2016,March,223,27115.66944,907555.3100,4510053.574,0.201229,0.008224,4069.754753,13.734294,2016-03-01,2016-03,0.201229,4069.754753,0.008224,0.000000,NaN
3,2016,April,222,26110.07392,517239.0800,4221943.049,0.122512,0.008502,2329.905766,12.922443,2016-04-01,2016-04,0.122512,2329.905766,0.008502,0.000000,NaN
4,2016,May,244,26759.23800,731777.8200,4270240.426,0.171367,0.009118,2999.089426,13.215158,2016-05-01,2016-05,0.171367,2999.089426,0.009118,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2021,June,346,41192.26835,782190.9300,5657087.373,0.138267,0.008400,2260.667428,10.937276,2021-06-01,2021-06,0.138267,2260.667428,0.008400,-0.001179,-0.000294
66,2021,July,390,44585.12587,587799.3414,5993902.054,0.098066,0.008747,1507.177798,10.883064,2021-07-01,2021-07,0.098066,1507.177798,0.008747,-0.000800,0.000967
67,2021,August,364,44703.31310,578387.7037,5894686.099,0.098120,0.008143,1588.977208,10.841560,2021-08-01,2021-08,0.098120,1588.977208,0.008143,-0.000747,-0.000510
68,2021,September,297,44148.45828,581239.8572,5833174.666,0.099644,0.006727,1957.036556,10.501785,2021-09-01,2021-09,0.099644,1957.036556,0.006727,-0.000643,-0.000393


In [34]:
data['final_rank'] = data['Trend'].rank(ascending=False)

In [35]:
#Flags outliers by StdDev of trend. Instead of Stddev from the mean, I chose to calculate based on Stddev from 0, 
#as 0 would indicate a flat trend, and the further from 0 the steeper the trend
mean = float(np.mean(data['Trend']))
std = float(np.std(data['Trend']))
data.loc[(data['Trend'])/std > upperBound, 'anomaly_score'] = 1
data.loc[(data['Trend'])/std < lowerBound, 'anomaly_score'] = 1
data.fillna(value = 0,inplace = True)
result = data[data['anomaly_score'] == 1]
print(mean)
print(std)
result

-0.00017726857929711344
0.0009675237851936326


,Accd_year,Accd_Month,claims_count,exposure,Ult_Claim_incurred,Ult_Earned_Premium,Attritional_Loss_Ratio,Claims_Frequency,Claims_Severity,Average_Premium,Month Name-Year,timestamp,ALR,Severity,Frequency,Trend,Change,final_rank,anomaly_score
24,2018,January,227,23764.03072,1.209005e+06,3815620.137,0.316857,0.009552,5326.014934,13.032647,2018-01-01,2018-01,0.316857,5326.014934,0.009552,-0.002666,0.002666,70.0,1.0
31,2018,August,263,27169.33776,5.346389e+05,3958069.264,0.135076,0.009680,2032.847414,11.915650,2018-08-01,2018-08,0.135076,2032.847414,0.009680,0.002244,-0.003077,1.0,1.0
53,2020,June,322,37661.58277,5.353382e+05,4794229.242,0.111663,0.008550,1662.540961,9.933941,2020-06-01,2020-06,0.111663,1662.540961,0.008550,-0.002055,0.001043,67.0,1.0
58,2020,November,376,40179.50018,6.603000e+05,4786778.904,0.137942,0.009358,1756.117091,9.337633,2020-11-01,2020-11,0.137942,1756.117091,0.009358,-0.002226,0.000171,69.0,1.0
59,2020,December,382,41722.36436,1.076387e+06,5116983.167,0.210356,0.009156,2817.767641,9.930259,2020-12-01,2020-12,0.210356,2817.767641,0.009156,-0.002193,0.000795,68.0,1.0


In [36]:
result.to_csv('Data/result.csv', header = True, index = False)

In [37]:
result["Premium_Score"] = result["Trend"] * result["Ult_Earned_Premium"]
result.sort_values(by=['Premium_Score'], ascending=False)

,Accd_year,Accd_Month,claims_count,exposure,Ult_Claim_incurred,Ult_Earned_Premium,Attritional_Loss_Ratio,Claims_Frequency,Claims_Severity,Average_Premium,Month Name-Year,timestamp,ALR,Severity,Frequency,Trend,Change,final_rank,anomaly_score,Premium_Score
31,2018,August,263,27169.33776,5.346389e+05,3958069.264,0.135076,0.009680,2032.847414,11.915650,2018-08-01,2018-08,0.135076,2032.847414,0.009680,0.002244,-0.003077,1.0,1.0,8881.988951
53,2020,June,322,37661.58277,5.353382e+05,4794229.242,0.111663,0.008550,1662.540961,9.933941,2020-06-01,2020-06,0.111663,1662.540961,0.008550,-0.002055,0.001043,67.0,1.0,-9851.936718
24,2018,January,227,23764.03072,1.209005e+06,3815620.137,0.316857,0.009552,5326.014934,13.032647,2018-01-01,2018-01,0.316857,5326.014934,0.009552,-0.002666,0.002666,70.0,1.0,-10173.248626
58,2020,November,376,40179.50018,6.603000e+05,4786778.904,0.137942,0.009358,1756.117091,9.337633,2020-11-01,2020-11,0.137942,1756.117091,0.009358,-0.002226,0.000171,69.0,1.0,-10655.906703
59,2020,December,382,41722.36436,1.076387e+06,5116983.167,0.210356,0.009156,2817.767641,9.930259,2020-12-01,2020-12,0.210356,2817.767641,0.009156,-0.002193,0.000795,68.0,1.0,-11219.779126
